## TV definition

The total variation $\text{TV}(f)$ of an image $f(x,y,z,t)$ is here defined using the $L_{2,1}$ norm as

\begin{equation}
\text{TV}(f) = \left\|  \nabla f  \right\|_{2,1} = \sum_{i,j,k,l} \sqrt{ (\partial_x f_l)_{i,j,k}^2 + (\partial_y f_l)_{i,j,k}^2  + (\partial_z f_l)_{i,j,k}^2 + \mu (\partial_t f_l)_{i,j,k}^2} \;,
\end{equation}

in which $\mu$ is a time-regularization factor, and $l$ is the time-index.

To compute the TV in practice, the image $f$ is discretized in space and time as a series of 3D image-vectors $\mathrm{f}=(\mathrm{f}_1, \dots, \mathrm{f}_{N_t})$.
Different choices of discretization can then be used to approximate the square of the spatial and time gradients.
For instance, the equations below correspond to an upwind discretization (1st order):

\begin{align}
(\partial_x \mathrm{f}_l)^2_{i,j,k} & = \left( (\mathrm{f}_l)_{i+1,j,k} - (\mathrm{f}_l)_{i,j,k} \right)^2 \\
(\partial_y \mathrm{f}_l)^2_{i,j,k} & = \left( (\mathrm{f}_l)_{i,j+1,k} - (\mathrm{f}_l)_{i,j,k} \right)^2 \\
(\partial_z \mathrm{f}_l)^2_{i,j,k} & = \left( (\mathrm{f}_l)_{i,j,k+1} - (\mathrm{f}_l)_{i,j,k} \right)^2 \\
(\partial_t \mathrm{f}_l)^2_{i,j,k} & = \left( (\mathrm{f}_{l+1})_{i,j,k} - (\mathrm{f}_l)_{i,j,k} \right)^2
\end{align}

Although this scheme is commonly used in TV calculations, it only provides first-order Taylor approximation of the estimated gradients, and can result in a certain alignment bias.
In PyTV-4D, four different discretizations are implemented, including a Hybrid Discretization Total Variation (HDTV) scheme that is both second order and mitigates the mentioned alignment bias.
The gradient discretization used in the four different schemes implemented are described below.

## General definitions

- 4D image-vector $f_{i,j,k,l}$ with $0 \leq (i,j) \leq N-1$, $0 \leq k \leq N_z-1$, and $0 \leq l \leq M-1$

- Row difference: $r_{i,j,k,l} = f_{i+1,j,k,l}-f_{i,j,k,l}$

- Column difference: $c_{i,j,k,l} = f_{i,j+1,k,l}-f_{i,j,k,l}$

- Slice difference: $s_{i,j,k,l} = f_{i,j,k+1,l}-f_{i,j,k,l}$

- Time difference: $t_{i,j,k,l} = f_{i,j,k,l+1}-f_{i,j,k,l}$

- Each direction is extended by 1 on both ends such that $r_{i,j,k,l}$, $c_{i,j,k,l}$, $s_{i,j,k,l}$, $t_{i,j,k,l}$ is defined for all $(i,j,k,l)$. Eg: we pick: $r_{N-1,j,k,l} = 0$ and $r_{-1,j,k,l} = 0$ for all $(j,k,l)$.

- 2D Example image-vector:

$
A = \begin{pmatrix}
0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 \\
\end{pmatrix}
$

## TV Upwind

- $TVU(f) = \sum_{l = 0}^{M-1} \sum_{k = 0}^{N_z-1} \sum_{i,j = 0}^{N-1} G^U_{i,j,k,l}$

- $G^U_{i,j,k,l} = \sqrt{r_{i,j,k,l}^2+c_{i,j,k,l}^2+s_{i,j,k,l}^2+t_{i,j,k,l}^2}$

- Not differentiable everywhere, isotropic.

- Subgradient NOT invariant by rotation.

- $[\partial TVU(f)]_{i,j,k,l} = \partial_{i,j,k,l}(G^U_{i,j,k,l}) + \partial_{i,j,k,l}(G^U_{i-1,j,k,l}) \\ + \partial_{i,j,k,l}(G^U_{i,j-1,k,l}) + \partial_{i,j,k,l}(G^U_{i,j,k-1,l}) + \partial_{i,j,k,l}(G^U_{i,j,k,l-1}) \\= - \frac{r_{i,j,k,l}+c_{i,j,k,l}+s_{i,j,k,l}+t_{i,j,k,l}}{G^U_{i,j,k,l}}+\frac{r_{i-1,j,k,l}}{G^U_{i-1,j,k,l}}+\frac{c_{i,j-1,k,l}}{G^U_{i,j-1,k,l}}+\frac{s_{i,j,k-1,l}}{G^U_{i,j,k-1,l}} + \frac{t_{i,j,k,l-1}}{G^U_{i,j,k,l-1}}$

- Example: $TVU(A) = 2+\sqrt{2}$ and the following image-vector is a subgradient:

$
\begin{pmatrix}
0 & 0 & 0 & 0 & 0 \\
0 & 0 & -1 & 0 & 0 \\
0 & -1 & 2+\sqrt{2} & -1/\sqrt{2} & 0 \\
0 & 0 & -1/\sqrt{2} & 0 & 0 \\
0 & 0 & 0 & 0 & 0 \\
\end{pmatrix}
$

## TV Downwind

- $TVD(f) = \sum_{l = 0}^{M-1} \sum_{k = 0}^{N_z-1} \sum_{i,j = 0}^{N-1} G^D_{i,j,k,l}$

- $G^D_{i,j,k,l} = \sqrt{r_{i-1,j,k,l}^2+c_{i,j-1,k,l}^2+s_{i,j,k-1,l}^2+t_{i,j,k,l-1}^2}$

- Not differentiable everywhere, isotropic.

- Subgradient NOT invariant by rotation.

- $[\partial TVD(f)]_{i,j,k,l} = \partial_{i,j,k,l}(G^D_{i,j,k,l}) + \partial_{i,j,k,l}(G^D_{i+1,j,k,l}) \\ + \partial_{i,j,k,l}(G^D_{i,j+1,k,l}) + \partial_{i,j,k,l}(G^D_{i,j,k+1,l}) + \partial_{i,j,k,l}(G^D_{i,j,k,l+1}) \\= \frac{r_{i-1,j,k,l}+c_{i,j-1,k,l}+s_{i,j,k-1,l}+t_{i,j,k,l-1}}{G^D_{i,j,k,l}}-\frac{r_{i,j,k,l}}{G^D_{i+1,j,k,l}}-\frac{c_{i,j,k,l}}{G^D_{i,j+1,k,l}}-\frac{s_{i,j,k,l}}{G^D_{i,j,k+1,l}}-\frac{t_{i,j,k,l }}{G^D_{i,j,k,l+1}}$

- Example: $TVD(A) = 2+\sqrt{2}$ and the following image-vector is a subgradient:

$
\begin{pmatrix}
0 & 0 & 0 & 0 & 0 \\
0 & 0 & -1 & 0 & 0 \\
0 & -1 & 2+\sqrt{2} & -1/\sqrt{2} & 0 \\
0 & 0 & -1/\sqrt{2} & 0 & 0 \\
0 & 0 & 0 & 0 & 0 \\
\end{pmatrix}
$

## TV Central

- $TVC(f) = \sum_{l = 0}^{M-1} \sum_{k = 0}^{N_z-1} \sum_{i,j = 0}^{N-1} G^C_{i,j,k,l}$

- $G^C_{i,j,k,l} = \sqrt{( \frac{r_{i,j,k,l}+r_{i-1,j,k,l}}{2})^2 + (\frac{c_{i,j,k,l} +c_{i,j-1,k,l}}{2})^2 \\  + (\frac{s_{i,j,k,l} +s_{i,j,k-1,l}}{2})^2  + (\frac{t_{i,j,k,l} +t_{i,j,k,l-1}}{2})^2 }$

- Not differentiable everywhere, isotropic. Checkerboard artifacts.

- Subgradient invariant by rotation.

- $[\partial TVC(f)]_{i,j,k,l} = \partial_{i,j,k,l}(G^C_{i-1,j,k,l}) + \partial_{i,j,k,l}(G^C_{i+1,j,k,l}) \\ + \partial_{i,j,k,l}(G^C_{i,j-1,k,l}) + \partial_{i,j,k,l}(G^C_{i,j+1,k,l}) + \partial_{i,j,k,l}(G^C_{i,j,k-1,l}) \\ + \partial_{i,j,k,l}(G^C_{i,j,k+1,l}) + \partial_{i,j,k,l}(G^C_{i,j,k,l-1}) + \partial_{i,j,k,l}(G^C_{i,j,k,l+1}) \\ = \frac{r_{i-1,j,k,l}+r_{i-2,j,k,l}}{G^C_{i-1,j,k,l}} - \frac{r_{i+1,j,k,l}+r_{i,j,k,l}}{G^C_{i+1,j,k,l}} +\frac{c_{i,j-1,k,l}+c_{i,j-2,k,l}}{G^C_{i,j-1,k,l}} - \frac{c_{i,j+1,k,l}+c_{i,j,k,l}}{G^C_{i,j+1,k,l}} \\ +\frac{s_{i,j,k-1,l}+s_{i,j,k-2,l}}{G^C_{i,j,k-1,l}} - \frac{s_{i,j,k+1,l}+s_{i,j,k,l}}{G^C_{i,j,k+1,l}} +\frac{t_{i,j,k,l-1}+s_{i,j,k,l-2}}{G^C_{i,j,k,l-1}} - \frac{t_{i,j,k,l+1}+t_{i,j,k,l}}{G^C_{i,j,k,l-1}}$

- $TVM(A) = 2.0$ and the following image-vector is a subgradient:

$
\begin{pmatrix}
0 & 0 & -0.5 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 \\
-0.5 & 0 & 2 & 0 & -0.5 \\
0 & 0 & 0 & 0 & 0 \\
0 & 0 & -0.5 & 0 & 0 \\
\end{pmatrix}
$

## TV Hybrid (Best)

- $TVH(f) = \sum_{l = 0}^{M-1} \sum_{k = 0}^{N_z-1} \sum_{i,j = 0}^{N-1} G^H_{i,j}$

- Not differentiable everywhere, isotropic.

- Subgradient invariant by rotation.

- $G^H_{i,j,k,l} = \sqrt{ \frac{r_{i,j,k,l}^2 + r_{i-1,j,k,l}^2}{2} + \frac{c_{i,j,k,l}^2 +c_{i,j-1,k,l}^2}{2} \\+ \frac{s_{i,j,k,l}^2 + s_{i,j,k-1,l}^2}{2} + \frac{t_{i,j,k,l}^2 + t_{i,j,k,l-1}^2}{2}}$

- $[\partial TVH(f)]_{i,j,k,l}$? Direct from combining both upwind and downwind terms.

- $TVH(A) = 3 \sqrt{2}$ and the following image-vector is a subgradient:

$
\begin{pmatrix}
0 & 0 & 0 & 0 & 0 \\
0 & 0 & -3\sqrt{2}/4 & 0 & 0 \\
0 & -3\sqrt{2}/4 & 3 \sqrt{2} & -3\sqrt{2}/4 & 0 \\
0 & 0 & -3\sqrt{2}/4 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 \\
\end{pmatrix}
$